In [1]:
# !pip install highway-env
# !pip install git+https://github.com/DLR-RM/stable-baselines3
# !pip install tensorboardx gym pyvirtualdisplay
# !apt-get install -y xvfb python-opengl ffmpeg
# !git clone https://github.com/eleurent/highway-env.git 2> /dev/null
# !git clone https://github.com/avivg7/highway-config.git
# !pip install tensorboard
# !pip install xvfbwrapper

In [1]:
from cffi import FFI
import os

ffi = FFI()
ffi.cdef("""
/* from stdio.h */
FILE* fopen(const char* path, const char* mode);
int fclose(FILE* fp);
FILE* stderr;  /* GNU C library */
FILE* __stderrp;  /* Mac OS X */
""")
try:
    stdio = ffi.dlopen(None)
    devnull = stdio.fopen(os.devnull.encode(), b'w')
except OSError:
    pass
try:
    stdio.stderr = devnull
except KeyError:
    try:
        stdio.__stderrp = devnull
    except KeyError:
        stdio.fclose(devnull)

/opt/conda/lib/python3.7/site-packages/cffi/cparser.py:165: UserWarning: Global variable 'stderr' in cdef(): for consistency with C it should have a storage class specifier (usually 'extern')
  "(usually 'extern')" % (decl.name,))
/opt/conda/lib/python3.7/site-packages/cffi/cparser.py:165: UserWarning: Global variable '__stderrp' in cdef(): for consistency with C it should have a storage class specifier (usually 'extern')
  "(usually 'extern')" % (decl.name,))


In [2]:
# General
import sys
sys.path.insert(0, '/content/highway-env/scripts/')
import io
import base64
import os
from tqdm.notebook import trange
import numpy as np
import random
import math
import pygame
import json
import ast
from pprint import pprint 
import glob
from collections import Counter

# Local
from final_project.display_utils import wrap_env, show_video
from models.agent import Agent
from models.utils import plot_learning_curve

# Gym Env
import gym
import highway_env
# from utils import record_videos, show_videos
from gym import logger as gymlogger
from gym.wrappers import Monitor
from gym.utils import seeding
from gym import error, spaces, utils
gymlogger.set_level(40) # error only

# Neural Networks
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Visualization
import matplotlib.pyplot as plt
from pyvirtualdisplay import Display
from IPython.display import HTML
from IPython import display as ipythondisplay

%load_ext tensorboard
%matplotlib inline

pygame 2.1.2 (SDL 2.0.16, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
display = Display(visible=0, size=(1400, 900))
display.start()

if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [4]:
# #=============== DO NOT DELETE ===============
# file = open('./highway-config/config_ex1.txt', 'r')
# contents = file.read()
# config1 = ast.literal_eval(contents)
# file.close()
# # ============================================

# env = gym.make("highway-fast-v0")
# env.configure(config1)

# obs = env.reset()
# for j in range(10):
#     obs, _, _, _ = env.step(0)

#     _, axes = plt.subplots(ncols=4, figsize=(12, 5))
#     for i, ax in enumerate(axes.flat):
#         ax.imshow(obs[i, ...].T, cmap=plt.get_cmap('gray'))
    

# env = wrap_env(env)
# env.reset()
# done = False
# iter = 0

# while (iter < 10) or not done:
#   if done:
#     break
#   iter +=1
#   action = env.action_space.sample()
#   observation, reward, done, _ = env.step(action)
#   screen = env.render(mode='rgb_array')
#   plt.imshow(screen)
#   print(f'iteration: {iter}, action: {action}, reward: {reward}, done: {done}')

In [5]:
torch.cuda.is_available()

True

In [6]:
#=============== DO NOT DELETE ===============
# file = open('/content/highway-config/config_ex1.txt', 'r')
# contents = file.read()
# config1 = ast.literal_eval(contents)
# file.close()
# ============================================

file = open('./highway-config/config_ex1.txt', 'r')
contents = file.read()
# print(contents)
config1 = ast.literal_eval(contents)
file.close()

env = gym.make("highway-fast-v0")
env.configure(config1)
# env = wrap_env(env)
env.reset()
# env.config

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[99., 99., 99., ..., 99., 99., 99.],
        [99., 99., 99., ..., 99., 99., 99.],
        [99., 99., 99., ..., 99., 99., 99.],
        ..

In [7]:
# a = np.max(observation)

# while a == 0:
#     action = env.action_space.sample()
#     observation, reward, done, _ = env.step(action)
#     a = np.max(observation)
# print(f'np.max(observation): {np.max(observation)}')
# print(f'np.min(observation): {np.min(observation)}')    
# for i in range(100):
#     action = env.action_space.sample()
#     observation, reward, done, _ = env.step(action)

#     pprint(action)
# # pprint(observation)
#     print(f'np.max(observation): {np.max(observation)}')
#     print(f'np.min(observation): {np.min(observation)}')

In [8]:
env.observation_space.shape

(4, 128, 128)

In [9]:
# del agent

In [ ]:
n_games = 500
figure_file = 'plots/highway_500'
torch.autograd.set_detect_anomaly(True)
# print(env.observation_space.shape)
agent = Agent(input_dims=env.observation_space.shape,
              env=env,
              n_actions=5,
              max_size=10000)

best_score = env.reward_range[0]
top_score = env.reward_range[0]
score_history = []
load_checkpoint = False
action_history = []

if load_checkpoint:
    agent.load_models()
    env.render(mode='human')
    
for i in range(n_games):
    observation = env.reset()
    done = False
    score = 0
    while not done:
        action = agent.choose_action(observation)
        action_history.append(action)
        if np.random.rand() > 0.15:
            new_observation, reward, done, info = env.step(action)
        else:
            new_observation, reward, done, info = env.step(env.action_space.sample())
        score += reward
        # print(f'observation.shape: {observation.shape}')
        agent.remember(observation, action, reward, new_observation, done)
        if not load_checkpoint:
            agent.learn()
        observation = new_observation
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    
    top_score = max(score, top_score)
    if avg_score > best_score:
        best_score = avg_score
        if not load_checkpoint:
            agent.save_models()
    
    screen = env.render(mode='rgb_array')
    plt.imshow(screen)
    print(f'Episode: {i}, Best Score {best_score}, Average Score: {avg_score}, Top Score: {top_score}')
    print(f'action_history: {np.histogram(action_history, bins=np.arange(5), density=True)}\n')
    
if not load_checkpoint:
    x = [i+1 for i in range(n_games)]
    plot_learning_curve(x, score_history, figure_file)

(10000, 4, 128, 128)
4 128 128


/home/jupyter/highway-rl/models/agent.py:58: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755953518/work/torch/csrc/utils/tensor_new.cpp:210.)
  state = T.Tensor([observation]).to(self.actor.device)
/home/jupyter/highway-rl/models/networks.py:137: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(prob)


Episode: 0, Best Score 19.050878684176222, Average Score: 19.050878684176222, Top Score: 19.050878684176222
action_history: (array([0.        , 0.31818182, 0.27272727, 0.40909091]), array([0, 1, 2, 3, 4]))

Episode: 1, Best Score 19.050878684176222, Average Score: 13.622748508541681, Top Score: 19.050878684176222
action_history: (array([0.03125, 0.34375, 0.3125 , 0.3125 ]), array([0, 1, 2, 3, 4]))

Episode: 2, Best Score 19.050878684176222, Average Score: 14.417367270355035, Top Score: 19.050878684176222
action_history: (array([0.06, 0.4 , 0.28, 0.26]), array([0, 1, 2, 3, 4]))

Episode: 3, Best Score 33.193585747787495, Average Score: 33.193585747787495, Top Score: 89.52224118008486
action_history: (array([0.12048193, 0.31325301, 0.21084337, 0.35542169]), array([0, 1, 2, 3, 4]))

Episode: 4, Best Score 33.193585747787495, Average Score: 28.773488750646912, Top Score: 89.52224118008486
action_history: (array([0.12290503, 0.30726257, 0.2122905 , 0.3575419 ]), array([0, 1, 2, 3, 4]))

Epi

In [ ]:
if not load_checkpoint:
    x = [i+1 for i in range(n_games)]
    plot_learning_curve(x, score_history, figure_file)

In [ ]:
# env.close()
# show_video()

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

    def init_weights(m):
        if isinstance(m, nn.Linear):
            torch.nn.init.uniform(m.weight,  -3*1.e-4, 3*1.e-4)
            m.bias.data.fill_(0.0001)

class ActorNet(nn.Module):
    def __init__(self):
        super(ActorNet, self).__init__()
        self.shared_layers = nn.Sequential( # todo: name
            nn.Conv2d(4, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(start_dim=0),
            nn.Linear(64*12*12, 5),
        )
        self.shared_layers.apply(init_weights)

    def forward(self, x):
        input_tensor = torch.tensor(x, dtype=torch.float)
        hidden = self.shared_layers(input_tensor)
        #output = self.output(hidden)
        actions_probs = Categorical(F.softmax(hidden, dim=-1))

        return actions_probs, hidden

class CriticNet(nn.Module):
    def __init__(self):
        super(CriticNet, self).__init__()
        self.shared_layers = nn.Sequential(  # todo: name
            nn.Conv2d(4, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(start_dim=0),
            nn.Linear(64 * 12 * 12, 1),
        )


    def forward(self, x):
        input_tensor = torch.FloatTensor(x)
        hidden = self.shared_layers(input_tensor)

        return hidden